<a href="https://colab.research.google.com/github/Apple03244/Colaboratory/blob/main/%ED%8D%BC%EC%85%89%ED%8A%B8%EB%A1%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import google.colab as colab

In [57]:
module=colab.files

In [58]:
#file_data=module.upload()

In [59]:
import pandas as pd
data=pd.read_csv('/content/train.csv')

In [60]:
use_data=data.dropna(axis=0)
use_data["일시"]=pd.to_datetime(use_data["일시"])

<ipython-input-60-089be084eef0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use_data["일시"]=pd.to_datetime(use_data["일시"])


In [61]:
target='평균기온'

## 1. 회귀를 위한 다중퍼셉트론(사이킷런)

In [62]:
import sklearn.neural_network as skneu
import sklearn.preprocessing as skpre
import sklearn.model_selection as skmod
import numpy as np
np.random.seed(10)
import sklearn.pipeline as skpip

In [63]:
mlp_model=skneu.MLPRegressor(hidden_layer_sizes=(50,50,50),random_state=10)
mlp_reg=skpip.make_pipeline(skpre.StandardScaler(),mlp_model)

In [64]:
train_data=use_data[use_data["일시"].apply(lambda x:x.year)<=2020]
test_data=use_data[use_data["일시"].apply(lambda x:x.year)>2020]

In [65]:
mlp_reg.fit(X=train_data[train_data.columns.difference(["일시",target])],y=train_data[target])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=(50, 50, 50),
                              random_state=10))])

In [66]:
import sklearn.metrics as skmet
pred_y=mlp_reg.predict(X=test_data[test_data.columns.difference(["일시",target])])
skmet.mean_squared_error(y_true=test_data[target],y_pred=pred_y,squared=False)

0.5648677372050859

## 2. 케라스를 이용한 다중 퍼셉트론

In [67]:
import tensorflow as ts

In [68]:
# 랜덤시드 고정
ts.random.set_seed(10)

### step 1. 모델 생성

In [69]:
model=ts.keras.Sequential() # 간단한 Sequential 모델을 만들거라 선언
model.add(ts.keras.layers.Input(shape=(9,))) # input shape 는
#ts.keras.Flatten 은 이미지를 1차원으로 전처리 해주는 충
model.add(ts.keras.layers.Dense(300,activation='relu'))
model.add(ts.keras.layers.Dense(100,activation='relu'))
model.add(ts.keras.layers.Dense(10,activation="relu"))
model.add(ts.keras.layers.Dense(1,activation="tanh"))

In [70]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 300)               3000      
                                                                 
 dense_13 (Dense)            (None, 100)               30100     
                                                                 
 dense_14 (Dense)            (None, 10)                1010      
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 34,121
Trainable params: 34,121
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.layers

### step 2. 모델 컴파일(평가기준)

In [82]:
model.compile(optimizer=ts.keras.optimizers.SGD(learning_rate=1e-6),
              loss=ts.keras.metrics.mean_absolute_error,
              metrics=["mae"]) #Adam은 역전파 알고리즘의 변형

### step 3. 모델 훈련 및 평가

In [84]:
train_feature=train_data.columns.difference(["일시",target])
model.fit(x=train_data[train_feature],y=train_data[target],
          epochs=100,validation_data=[test_data[train_feature],test_data[target]])

Epoch 1/100
212/212 [==============================] - 2s 10ms/step - loss: 14.1936 - mae: 14.1936 - val_loss: 15.3398 - val_mae: 15.3398
Epoch 2/100
212/212 [==============================] - 1s 5ms/step - loss: 14.1854 - mae: 14.1854 - val_loss: 15.3326 - val_mae: 15.3326
Epoch 3/100
212/212 [==============================] - 1s 3ms/step - loss: 14.1785 - mae: 14.1785 - val_loss: 15.3265 - val_mae: 15.3265
Epoch 4/100
212/212 [==============================] - 1s 2ms/step - loss: 14.1726 - mae: 14.1726 - val_loss: 15.3213 - val_mae: 15.3213
Epoch 5/100
212/212 [==============================] - 1s 3ms/step - loss: 14.1676 - mae: 14.1676 - val_loss: 15.3168 - val_mae: 15.3168
Epoch 6/100
212/212 [==============================] - 1s 2ms/step - loss: 14.1632 - mae: 14.1632 - val_loss: 15.3129 - val_mae: 15.3129
Epoch 7/100
212/212 [==============================] - 1s 3ms/step - loss: 14.1592 - mae: 14.1592 - val_loss: 15.3095 - val_mae: 15.3095
Epoch 8/100
212/212 [===================